In [1]:
import pandas as pd
import spacy
import re
from tqdm import tqdm
from utils import word_tag_pairs
from utils import extract_email_msg
from utils import imperative_sentences
nlp = spacy.load('en_core_web_sm')

In [3]:
data = pd.read_csv(r'emails.csv',encoding = 'utf-8-sig')
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
contents = extract_email_msg(data)
data["email_contents"] = contents
print(len(contents))

100%|████████████████████████████████████████████████████████████████████████| 517401/517401 [01:13<00:00, 7005.06it/s]


517401


In [5]:
results = []
for text in tqdm(data["email_contents"].tolist()):
    temp = re.sub('\n', '', text)
    doc = nlp(temp)
    results.append(imperative_sentences(word_tag_pairs(doc)))

 19%|█████████████                                                         | 96729/517401 [1:50:12<12:04:50,  9.67it/s]

ValueError: [E088] Text of length 1985271 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

 19%|█████████████                                                         | 96729/517401 [1:50:31<12:04:50,  9.67it/s]

In [7]:
len(results)
results[:10]

[False, False, False, True, True, False, True, False, False, False]

In [ ]:
data["contains_actionable"] = results

In [10]:
df = pd.DataFrame()
df['contents'] = contents[:len(results)]
df['results'] = results

df.shape

(96731, 2)

In [11]:
# df.to_csv(r'obj_1.csv',encoding='utf-8-sig',index = False)

In [ ]:
-----------------------------------------------PREPARING THE DATASETS-----------------------------------------------------

In [10]:
df = pd.read_csv(r'actions.csv',encoding='utf-8-sig',header = None)
list_action = df[0].tolist()

In [15]:
len(list_action)

1250

In [21]:
list_action_1 = []
for i in list_action:
    list_action_1.append(i.lower())

In [5]:
data_1 = pd.read_csv(r'obj_1.csv',encoding='utf-8-sig')
data_2 = data_1.loc[data_1['results'] == False]
data_2.head()

,contents,results
0,here is our forecast\n\n,False
1,traveling to have a business meeting takes the...,False
2,test successful. way to go!!!,False
5,"greg,\n\n how about either next tuesday or thu...",False
7,any morning between 10 and 11:30,False


In [6]:
list_1 = data_2['contents'].tolist()
len(list_1)

82363

In [17]:
## Get only certain length Sentences
count = 0
list_nonaction = []
for j in list_1:
    if count < 1300:
        j = re.sub('\n', '', j)
        if len(j) < 100:
            list_nonaction.append(j)
            count += 1

In [18]:
len(list_nonaction)

1300

In [22]:
list_nonaction[:50]

['activate all who work with transmission or have any good ideas on the subject.',
 'add more to your score by stopping in and picking up hefty load of construction supplies to win.',
 "add o'neal winfee and george smith to the attendees list.",
 'additionally, send me the payment schedule for tenaska iv this month.',
 'adjust our purchase amount from each party based on the transport allocation.',
 'after the next meeting, let me know on the timing of preparing this draft.',
 'after this is done please let stephanie know so that she can redraft her invoice the next day after pops bridges to mops.',
 'alex, please use the second check as the october payment.',
 'alicia, please schedule me to attend this meeting.',
 'allen please authorize the following products for app',
 'allen please authorize the following products for approval.',
 'allocate them evenly among these teams.',
 'allow people to know month ahead if they will be affected by rolling blackouts.',
 'also include the increme

In [23]:
dff = pd.DataFrame()
dff['sentences'] = list_nonaction+list_action_1

dff.to_csv(r'Actionable_Binary_Classify.csv',encoding = 'utf-8-sig',index = False)